# 5.0 — Diebold–Mariano Test (GARCH vs EWM)




In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
from scipy.stats import t as student_t

IN = Path("data/models/garch/garch_forecasts.csv")


### 5.1 Fonction Diebold–Mariano test

**Explication :**  
1) La fonction `dm_test` compare la précision de deux modèles de prévision en utilisant la statistique de Diebold–Mariano (DM).  
2) On calcule la différence des erreurs quadratiques (MSE) entre les deux modèles.  
3) On estime la variance avec Newey–West.  
4) On applique la correction de Harvey pour les petits échantillons.  
5) La fonction retourne la statistique DM ajustée et la p-value associée.  

**Pourquoi nous avons fait cela :**  
1) Le test de Diebold–Mariano est la référence pour comparer statistiquement les performances de deux modèles de prévision.  
2) Il permet de déterminer si les différences observées en termes d’erreur de prévision sont significatives ou simplement dues au hasard.  


In [ ]:
def dm_test(e1, e2, h=1):
    """Diebold-Mariano test (Harvey et al., small-sample correction) sur MSE."""
    d = (e1**2 - e2**2)  
    T = len(d)
    dbar = d.mean()
    s2 = d.var(ddof=1)
    DM = dbar / np.sqrt(s2 / T)
    k = ((T + 1 - 2*h + h*(h-1)/T) ** 0.5)
    DM_adj = DM * (T / (T - 1)) ** 0.5 * k
    p = 2 * (1 - student_t.cdf(abs(DM_adj), df=T-1))
    return DM_adj, p


### 5.2 Fonction principale : comparaison GARCH vs EWM avec le test DM

**Explication :**  
1) On charge les données depuis un fichier CSV.  
2) On définit les erreurs de prévision des modèles GARCH et EWM par rapport à la volatilité réalisée.  
3) On applique le test de Diebold–Mariano pour comparer statistiquement la précision des deux modèles.  
4) On affiche la statistique DM, la p-value et le modèle qui est significativement meilleur si p < 0.05.  

**Pourquoi nous avons fait cela :**  
1) Cela permet de tester objectivement si GARCH apporte une amélioration par rapport à EWM en termes de précision de prévision.  
2) L’interprétation statistique (p-value) garantit que la conclusion n’est pas due au hasard mais validée par un test reconnu en économétrie.  


In [3]:
def main():
    df = pd.read_csv(IN, parse_dates=["date"])
    y = df["vol_real"].values
    g = df["vol_garch"].values
    e = df["vol_ewm"].values

    e1 = g - y
    e2 = e - y

    stat, p = dm_test(e1, e2, h=1)
    print("Diebold-Mariano (MSE) GARCH vs EWM")
    print("  DM stat =", round(float(stat), 3), "| p-value =", round(float(p), 4))
    if p < 0.05:
        better = "GARCH" if np.mean(e1**2) < np.mean(e2**2) else "EWM"
        print(f"  ⇒ Différence significative. Meilleur: {better}.")
    else:
        print("  ⇒ Pas de différence significative au seuil 5%.")
    

In [4]:
if __name__ == "__main__":
    main()


Diebold-Mariano (MSE) GARCH vs EWM
  DM stat = 1255.828 | p-value = 0.0
  ⇒ Différence significative. Meilleur: EWM.
